In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
from tqdm import tqdm
import gc
import pyarrow.parquet as pq

In [ ]:
pip install mljar-supervised

In [ ]:
from supervised.preprocessing.eda import EDA

In [ ]:
def rm(df):
    for col in df.columns:
        col_type = df[col].dtype        
        if (col_type != object) and (type(col_type) != pd.core.dtypes.dtypes.CategoricalDtype):
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')                    
    return df
def r(path):
    return rm(pq.read_table(path).to_pandas())
def rf(file_path):
    file_list = glob.glob(file_path + '/*')
    file = r(file_list[0])
    if "stock_id" not in file.columns: 
        file["stock_id"] = int(file_list[0].split("/")[-1].split("=")[-1])
    for i in tqdm(range(1, len(file_list))):
        data2 = r(file_list[i])
        if "stock_id" not in data2.columns:
            data2["stock_id"] = int(file_list[i].split("/")[-1].split("=")[-1])
        file = pd.concat([file, data2])
        del data2
        gc.collect()
    file = file.reset_index(drop = True)
    return file

# Trade train

In [ ]:
trade_train = rf('../input/optiver-realized-volatility-prediction/trade_train.parquet')
trade_train

In [ ]:
trade_train.to_csv('trade_train.csv', index=False)

# Book train

In [ ]:
book_train = rf("../input/optiver-realized-volatility-prediction/book_train.parquet")
book_train

In [ ]:
book_train[book_train.stock_id == 0]

100000000+ rows?, ok

In [ ]:
book_train.to_csv('book_train.csv', index=False)

<a href="./book_train.csv"> Download File </a>

# Trade test

In [ ]:
trade_test = rf("../input/optiver-realized-volatility-prediction/trade_test.parquet")
trade_test

# Book test

In [ ]:
book_test = rf("../input/optiver-realized-volatility-prediction/book_test.parquet")
book_test

In [ ]:
train = pd.read_csv("../input/optiver-realized-volatility-prediction/train.csv")
train

In [ ]:
EDA.extensive_eda(train[["stock_id", "time_id"]], train.target, save_path="./")

# If the content is helpful, please upvote. :)